
# 🟦 RAG Hands-On (Node.js) — Company Files Q&A Chatbot (Backend Only)

**Goal:** Sketch a **clean Node.js RAG backend** for Q&A over company files (PDF, DOCX, CSV):

- No UI, no Tavily, no web search — just a backend-style pipeline.  
- Uses **popular models & tools** via LangChain.js-style patterns.  
- Offline-friendly vector stores (FAISS / Memory / Chroma client).  
- Clear structure so you can turn this into a real Node project.

> This notebook is mostly **Markdown + code snippets**.  
> Copy the code into your Node project (`src/` folder) and run there.



## 0. Project Structure (suggested)

```bash
rag-node-backend/
  package.json
  tsconfig.json           # if using TypeScript
  src/
    config.ts
    loaders/
      pdfLoader.ts
      docxLoader.ts
      csvLoader.ts
    chunking.ts
    embeddings.ts
    vectorstores.ts
    llms.ts
    memory.ts
    rag.ts
    server.ts             # optional HTTP server for testing
  data/
    ... your PDF/DOCX/CSV files ...
```

You can keep it all in one file at the beginning (e.g. `rag.ts`),  
then refactor into modules as it grows.



## 1. `package.json` (Core Dependencies)

Example for **TypeScript + Node**:

```jsonc
{
  "name": "rag-node-backend",
  "version": "1.0.0",
  "type": "module",
  "main": "dist/server.js",
  "scripts": {
    "dev": "ts-node src/server.ts",
    "build": "tsc",
    "start": "node dist/server.js"
  },
  "dependencies": {
    "langchain": "^0.3.0",
    "@langchain/community": "^0.3.0",
    "@langchain/openai": "^0.3.0",
    "faiss-node": "^0.5.0",
    "chromadb": "^1.8.0",
    "pdf-parse": "^1.1.1",
    "docx": "^9.0.0",
    "csv-parse": "^5.5.0",
    "dotenv": "^16.0.0"
  },
  "devDependencies": {
    "typescript": "^5.0.0",
    "ts-node": "^10.0.0"
  }
}
```

> Versions are illustrative — adjust to current ones when you actually install.



## 2. Config & Environment (`src/config.ts`)

```ts
// src/config.ts
import * as dotenv from "dotenv";
dotenv.config();

export const DATA_DIR = "./data";

export const OPENAI_API_KEY = process.env.OPENAI_API_KEY || "";

// Choose default models
export const DEFAULT_CHAT_MODEL = "gpt-4o-mini"; // or your available model
export const DEFAULT_EMBEDDING_MODEL = "text-embedding-3-small";
```


## 3. File Loaders (`src/loaders/*`)

Simplified approach using common Node libs.



### `src/loaders/pdfLoader.ts`

```ts
import fs from "fs";
import path from "path";
import pdfParse from "pdf-parse";

export async function loadPdf(filePath: string) {
  const dataBuffer = fs.readFileSync(filePath);
  const pdfData = await pdfParse(dataBuffer);
  // Simple: one big doc; you can also split by page if needed
  return [
    {
      pageContent: pdfData.text,
      metadata: { source: filePath },
    },
  ];
}
```


### `src/loaders/docxLoader.ts`

```ts
import fs from "fs";
import { Document, Packer } from "docx"; // docx parsing is a bit tricky; pseudo-code here

export async function loadDocx(filePath: string) {
  // In practice, consider using a higher-level docx-to-text library
  const buffer = fs.readFileSync(filePath);
  // Placeholder: you'd replace this with actual DOCX text extraction
  const text = buffer.toString("utf8");

  return [
    {
      pageContent: text,
      metadata: { source: filePath },
    },
  ];
}
```


### `src/loaders/csvLoader.ts`

```ts
import fs from "fs";
import { parse } from "csv-parse/sync";

export async function loadCsv(filePath: string, textCols?: string[]) {
  const content = fs.readFileSync(filePath, "utf8");
  const records = parse(content, { columns: true, skip_empty_lines: true });

  const docs = records.map((row: any, idx: number) => {
    const cols = textCols || Object.keys(row);
    const pieces = cols
      .filter((col) => row[col] !== undefined && row[col] !== null)
      .map((col) => `${col}: ${row[col]}`);
    const text = pieces.join("\n");
    return {
      pageContent: text,
      metadata: { source: filePath, row: idx },
    };
  });

  return docs;
}
```


### Aggregate loader (`src/loaders/index.ts`)

```ts
// src/loaders/index.ts
import fs from "fs";
import path from "path";
import { loadPdf } from "./pdfLoader";
import { loadDocx } from "./docxLoader";
import { loadCsv } from "./csvLoader";

export interface LCSDocument {
  pageContent: string;
  metadata: Record<string, any>;
}

export async function loadAllDocuments(dataDir: string): Promise<LCSDocument[]> {
  const docs: LCSDocument[] = [];

  function walk(dir: string) {
    const entries = fs.readdirSync(dir, { withFileTypes: true });
    for (const entry of entries) {
      const full = path.join(dir, entry.name);
      if (entry.isDirectory()) walk(full);
      else {
        if (entry.name.toLowerCase().endsWith(".pdf")) {
          // PDF
        } else if (entry.name.toLowerCase().endsWith(".docx")) {
          // DOCX
        } else if (entry.name.toLowerCase().endsWith(".csv")) {
          // CSV
        }
      }
    }
  }

  const entries = fs.readdirSync(dataDir, { withFileTypes: true });
  for (const entry of entries) {
    const full = path.join(dataDir, entry.name);
    if (entry.isDirectory()) {
      docs.push(...(await loadAllDocuments(full)));
    } else if (entry.name.toLowerCase().endsWith(".pdf")) {
      docs.push(...(await loadPdf(full)));
    } else if (entry.name.toLowerCase().endsWith(".docx")) {
      docs.push(...(await loadDocx(full)));
    } else if (entry.name.toLowerCase().endsWith(".csv")) {
      docs.push(...(await loadCsv(full)));
    }
  }

  return docs;
}
```


## 4. Chunking (`src/chunking.ts`)

LangChain.js provides text splitters similar to Python.



```ts
// src/chunking.ts
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
import type { LCSDocument } from "./loaders";

export async function chunkDocuments(
  docs: LCSDocument[],
  chunkSize = 1000,
  chunkOverlap = 200
): Promise<LCSDocument[]> {
  const splitter = new RecursiveCharacterTextSplitter({
    chunkSize,
    chunkOverlap,
    separators: ["\n\n", "\n", ". ", " ", ""],
  });

  const allChunks: LCSDocument[] = [];
  for (const d of docs) {
    const split = await splitter.splitText(d.pageContent);
    split.forEach((text) => {
      allChunks.push({
        pageContent: text,
        metadata: d.metadata,
      });
    });
  }

  return allChunks;
}
```


## 5. Embeddings & Vector Stores (`src/embeddings.ts`, `src/vectorstores.ts`)

We define multiple embedding models + vector stores.

- Embeddings:
  - OpenAI embeddings
  - Local model (e.g., `@langchain/community/embeddings/hf`) — placeholder here
- Vector stores:
  - FAISS (via `faiss-node` or LangChain bindings)
  - Memory store for quick testing
  - Chroma (requires running a Chroma server)



```ts
// src/embeddings.ts
import { OpenAIEmbeddings } from "@langchain/openai";
// You can also import HF embeddings if desired:
// import { HuggingFaceInferenceEmbeddings } from "@langchain/community/embeddings/hf";

export type EmbeddingName = "openai_small" | "openai_large";

export function getEmbeddingModel(name: EmbeddingName) {
  if (name === "openai_small") {
    return new OpenAIEmbeddings({
      model: "text-embedding-3-small",
    });
  }
  if (name === "openai_large") {
    return new OpenAIEmbeddings({
      model: "text-embedding-3-large",
    });
  }
  throw new Error(`Unknown embedding model: ${name}`);
}
```


```ts
// src/vectorstores.ts
import { MemoryVectorStore } from "langchain/vectorstores/memory";
import { FaissStore } from "@langchain/community/vectorstores/faiss";
import { ChromaClient } from "chromadb";
import type { EmbeddingsInterface } from "@langchain/core/embeddings";
import type { LCSDocument } from "./loaders";

export type VectorStoreName = "memory" | "faiss" | "chroma";

export async function buildVectorStore(
  chunks: LCSDocument[],
  storeName: VectorStoreName,
  embeddings: EmbeddingsInterface
) {
  if (storeName === "memory") {
    return await MemoryVectorStore.fromTexts(
      chunks.map((c) => c.pageContent),
      chunks.map((c) => c.metadata),
      embeddings
    );
  }

  if (storeName === "faiss") {
    return await FaissStore.fromTexts(
      chunks.map((c) => c.pageContent),
      chunks.map((c) => c.metadata),
      embeddings
    );
  }

  if (storeName === "chroma") {
    const client = new ChromaClient({ path: "http://localhost:8000" });
    const collectionName = "company_docs";
    const collection = await client.getOrCreateCollection({ name: collectionName });
    // You would then upsert docs into Chroma here (ids, embeddings, metadata).
    // For brevity, we skip full implementation.
    return collection;
  }

  throw new Error(`Unknown vector store: ${storeName}`);
}
```


## 6. Chat Models & Memory (`src/llms.ts`, `src/memory.ts`)

Use several chat models (by name) and a simple in-memory conversation buffer.



```ts
// src/llms.ts
import { ChatOpenAI } from "@langchain/openai";

export type ChatModelName = "gpt_4_small" | "gpt_4_full";

export function getChatModel(name: ChatModelName) {
  if (name === "gpt_4_small") {
    return new ChatOpenAI({
      model: "gpt-4o-mini",
      temperature: 0,
    });
  }
  if (name === "gpt_4_full") {
    return new ChatOpenAI({
      model: "gpt-4.1",
      temperature: 0,
    });
  }
  throw new Error(`Unknown chat model: ${name}`);
}
```


```ts
// src/memory.ts
export interface ConversationTurn {
  user: string;
  assistant: string;
}

export class SimpleConversationMemory {
  private maxTurns: number;
  private history: ConversationTurn[];

  constructor(maxTurns = 5) {
    this.maxTurns = maxTurns;
    this.history = [];
  }

  addTurn(user: string, assistant: string) {
    this.history.push({ user, assistant });
    if (this.history.length > this.maxTurns) {
      this.history.shift();
    }
  }

  formatHistory(): string {
    return this.history
      .map(
        (h) =>
          `User: ${h.user}\nAssistant: ${h.assistant}`
      )
      .join("\n");
  }
}
```


## 7. RAG Pipeline (`src/rag.ts`)

Tie everything together:

- Load & chunk docs,
- Build vector store,
- Retrieve top-k chunks,
- Build prompt with context + history,
- Call chat model,
- Update memory.



```ts
// src/rag.ts
import { loadAllDocuments } from "./loaders";
import { chunkDocuments } from "./chunking";
import { getEmbeddingModel, EmbeddingName } from "./embeddings";
import { buildVectorStore, VectorStoreName } from "./vectorstores";
import { getChatModel, ChatModelName } from "./llms";
import { SimpleConversationMemory } from "./memory";
import { DATA_DIR } from "./config";
import { HumanMessage, SystemMessage } from "@langchain/core/messages";

export class RAGPipeline {
  private embeddingName: EmbeddingName;
  private vsName: VectorStoreName;
  private chatName: ChatModelName;
  private memory: SimpleConversationMemory;
  private vectorStore: any;
  private chatModel: any;

  constructor(
    embeddingName: EmbeddingName = "openai_small",
    vsName: VectorStoreName = "memory",
    chatName: ChatModelName = "gpt_4_small"
  ) {
    this.embeddingName = embeddingName;
    this.vsName = vsName;
    this.chatName = chatName;
    this.memory = new SimpleConversationMemory(5);
  }

  async init() {
    const rawDocs = await loadAllDocuments(DATA_DIR);
    const chunks = await chunkDocuments(rawDocs);

    const embeddings = getEmbeddingModel(this.embeddingName);
    this.vectorStore = await buildVectorStore(chunks, this.vsName, embeddings);
    this.chatModel = getChatModel(this.chatName);
  }

  private buildContextFromDocs(docs: any[]): string {
    return docs
      .map((d, idx) => {
        const src = d.metadata?.source || "unknown";
        return `[${idx + 1} | ${src}]\n${d.pageContent}`;
      })
      .join("\n---\n");
  }

  async answer(question: string, k = 5): Promise<string> {
    // 1) Retrieve
    const docs = await this.vectorStore.similaritySearch(question, k);
    const contextText = this.buildContextFromDocs(docs);

    const historyText = this.memory.formatHistory();
    const historyBlock = historyText
      ? `\n\nConversation so far:\n${historyText}`
      : "";

    const systemPrompt =
      "You are a helpful assistant answering questions based only on the provided context. " +
      "If the answer is not in the context, say you don't know.";

    const userContent = `Context:\n${contextText}\n\n${historyBlock}\n\nUser question: ${question}\n\nAnswer:`;

    const messages = [
      new SystemMessage(systemPrompt),
      new HumanMessage(userContent),
    ];

    const response = await this.chatModel.invoke(messages);
    const answer = response.content as string;

    this.memory.addTurn(question, answer);
    return answer;
  }
}
```


## 8. Simple Test Server (`src/server.ts`)

Optional: expose `/ask` endpoint for quick manual testing.



```ts
// src/server.ts
import http from "http";
import { RAGPipeline } from "./rag";

async function main() {
  const rag = new RAGPipeline("openai_small", "memory", "gpt_4_small");
  await rag.init();

  const server = http.createServer(async (req, res) => {
    if (req.method === "POST" && req.url === "/ask") {
      let body = "";
      req.on("data", (chunk) => (body += chunk.toString()));
      req.on("end", async () => {
        try {
          const { question } = JSON.parse(body);
          const answer = await rag.answer(question, 5);
          res.writeHead(200, { "Content-Type": "application/json" });
          res.end(JSON.stringify({ answer }));
        } catch (err: any) {
          res.writeHead(500, { "Content-Type": "application/json" });
          res.end(JSON.stringify({ error: err.message }));
        }
      });
    } else {
      res.writeHead(404);
      res.end("Not found");
    }
  });

  const port = 3000;
  server.listen(port, () => {
    console.log(`RAG server listening on http://localhost:${port}`);
  });
}

main().catch((err) => {
  console.error("Failed to start server:", err);
});
```


---
### ✅ Summary (Node.js Hands-On)

This notebook gave you a **backend-oriented RAG design in Node.js**:

- Loaders for **PDF / DOCX / CSV**,  
- Chunking with `RecursiveCharacterTextSplitter`,  
- Multiple embeddings (OpenAI; extendable to HF),  
- Multiple vector stores (Memory, FAISS, Chroma client),  
- Configurable chat models,  
- Simple conversation memory,  
- A `RAGPipeline` class + optional HTTP server.

Use this as your **blueprint** to implement a real Node.js RAG backend for company Q&A.
